In [47]:
import os

os.chdir("..")

from automation.workflows import AmazonAccountCreationWorkflow

In [48]:
# amazon_account_creation_flow = AmazonAccountCreationWorkflow()

In [75]:
from automation.base_driver import BaseDriver
from automation.pages.amazon_homepage import AmazonHomePage
from automation.pages.amazon_signup_page import AmazonSignupPage
from automation.pages.amazon_profile_page import AmazonProfilePage
from automation.pages.amazon_developer_registration import AmazonDevRegistrationPage
from integrations.virtual_number_service import OnlineSimService
from integrations.tempmail_service import TempMailService
from utils.logger import logger
from utils.utils import get_fake_name, get_fake_password

self = BaseDriver()

self.name = "John Dear"
self.email = None
self.password = None
self.phone_number = None
self.totp_secret = None
self.country = None or "canada"
self.driver = self.get_driver(enable_captcha_solver=True, enable_proxy=True)

In [76]:
temp_mail_service = TempMailService()
self.email = temp_mail_service.get_new_email_id()

self.name = get_fake_name()
self.password = get_fake_password()
logger.info(f"Generated new phone number: {self.phone_number}")

home_page = AmazonHomePage(driver=self.driver)

2025-02-26 00:08:47,055 - INFO - Generated new phone number: None


In [77]:
home_page.load_page()
home_page.get_signup_page()

LinkTextNotFoundException: 
 Link text {Create Developer Account} was not found after 7 seconds!

In [88]:
home_page.driver.find_elements(By.TAG_NAME, "a")

[]

In [90]:
home_page.driver.save_screenshot("test.png")

C:\Users\Anon\PycharmProjects\appstore-automation\.venv\lib\site-packages\selenium\webdriver\remote\webdriver.py:843: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  return self.get_screenshot_as_file(filename)


TimeoutException: Message: timeout: Timed out receiving message from renderer: 10.000
  (Session info: chrome=133.0.6943.127)
Stacktrace:
	GetHandleVerifier [0x00007FF64E086F15+28773]
	(No symbol) [0x00007FF64DFF2600]
	(No symbol) [0x00007FF64DE88FAA]
	(No symbol) [0x00007FF64DE7646C]
	(No symbol) [0x00007FF64DE7615A]
	(No symbol) [0x00007FF64DE73CEE]
	(No symbol) [0x00007FF64DE747AF]
	(No symbol) [0x00007FF64DE99E10]
	(No symbol) [0x00007FF64DF3F0B9]
	(No symbol) [0x00007FF64DF0724A]
	(No symbol) [0x00007FF64DF2F6F3]
	(No symbol) [0x00007FF64DF07023]
	(No symbol) [0x00007FF64DECFF5E]
	(No symbol) [0x00007FF64DED11E3]
	GetHandleVerifier [0x00007FF64E3D425D+3490733]
	GetHandleVerifier [0x00007FF64E3EBA43+3586963]
	GetHandleVerifier [0x00007FF64E3E147D+3544525]
	GetHandleVerifier [0x00007FF64E14C9DA+838442]
	(No symbol) [0x00007FF64DFFD04F]
	(No symbol) [0x00007FF64DFF9614]
	(No symbol) [0x00007FF64DFF97B6]
	(No symbol) [0x00007FF64DFE8CE9]
	BaseThreadInitThunk [0x00007FFF65E255A0+16]
	R

In [ ]:
signup_page = AmazonSignupPage(driver=self.driver)
signup_page.fill_signup_form(
    name=self.name, email=self.email, phone_number=self.phone_number, password=self.password,
    country_name=self.country
)


In [ ]:
self.email

In [ ]:
otp = None

# try 3 times to get otp
logger.info(f"Waiting for email otp to arrive from amazon")
for retry in range(3):
    otp = temp_mail_service.get_amazon_message()
    # otp = online_sim.get_message_with_wait(tzid=tzid)
    if otp is not None:
        break

    logger.info(f"No OTP in email, Retrying again: retry count: {retry}")
    self.random_sleep()
    signup_page.resend_otp()

if otp is None:
    logger.info("Did not receive email otp after retrying 3 times, skipping this automation")
signup_page.enter_otp(code=otp)

In [59]:
# phone verification logic
online_sim = OnlineSimService()
tzid = online_sim.get_new_number()  # default number will be from canada
self.phone_number = online_sim.get_number(tzid=tzid)


In [40]:
signup_page.enter_mobile_number(country_name=self.country, phone_number=self.phone_number)
signup_page.submit_form()

logger.info(f"Waiting for phone otp to arrive from amazon")
for retry in range(3):
    otp = online_sim.get_message_with_wait(tzid=tzid)
    if otp is not None:
        break

    logger.info(f"No OTP in phone, Retrying again: retry count: {retry}")
    self.random_sleep()
    signup_page.resend_otp()

if otp is None:
    logger.info("Did not receive email otp after retrying 3 times, skipping this automation")
    return

signup_page.enter_otp(code=otp)

NoSuchElementException: Message: 
 Element {#ap_email} was not present after 10 seconds!


In [44]:
# self.driver.refresh()


In [60]:
from selenium.webdriver.common.by import By

country_name = "canada"
self.driver.click('//span[@class="a-button-text a-declarative"]', "xpath")
country_dropdown = self.driver.find_elements(By.TAG_NAME, "a")
for element in country_dropdown:
    if element.text and country_name.lower() in element.text.lower():
        element.click()
        logger.debug(f"Selected country {country_name} from phone dropdown")
        break


In [61]:
phone_number = self.phone_number

class AmazonSignupPageLocators:
    NEW_ACCOUNT_BTN = "#createAccountSubmit"
    EMAIL_INPUT = "#ap_email"
    USERNAME_INPUT = "#ap_customer_name"
    PASSWORD_INPUT = "#ap_password"
    PASSWORD_CHECK_INPUT = "#ap_password_check"
    PHONE_NUMBER_INPUT = "#cvfPhoneNumber"
    COUNTRY_DROPDOWN = "//span[@class='a-button-text a-declarative']"
    SUBMIT_BUTTON = "#continue"
    VERIFICATION_CODE_INPUT = "#cvf-input-code"
    SUBMIT_VERIFICATION = "//input[@aria-label='Verify OTP Button']"
    RESEND_VERIFICATION = "cvf-resend-link"
    EXISTING_ACCOUNT_MSG = "//h4[text()='Mobile number already in use']"

self.driver.type(AmazonSignupPageLocators.PHONE_NUMBER_INPUT, phone_number)

In [62]:
self.driver.click("//input[@type='submit']", "xpath")

In [65]:

logger.info(f"Waiting for phone otp to arrive from amazon")
for retry in range(3):
    otp = online_sim.get_message_with_wait(tzid=tzid)
    if otp is not None:
        break

    logger.info(f"No OTP in phone, Retrying again: retry count: {retry}")
    self.random_sleep()
    signup_page.resend_otp()

if otp is None:
    logger.info("Did not receive email otp after retrying 3 times, skipping this automation")
    return

signup_page.enter_otp(code=otp)
self.random_sleep()
signup_page.submit_form()


2025-02-25 23:55:00,912 - INFO - Waiting for phone otp to arrive from amazon
2025-02-25 23:55:29,989 - INFO - Did not receive otp in 30 seconds
2025-02-25 23:55:29,998 - INFO - No OTP in phone, Retrying again: retry count: 0


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="cvf-resend-link"]"}
  (Session info: chrome=133.0.6943.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF64E086F15+28773]
	(No symbol) [0x00007FF64DFF2600]
	(No symbol) [0x00007FF64DE88FAA]
	(No symbol) [0x00007FF64DEDF286]
	(No symbol) [0x00007FF64DEDF4BC]
	(No symbol) [0x00007FF64DF32A27]
	(No symbol) [0x00007FF64DF0728F]
	(No symbol) [0x00007FF64DF2F6F3]
	(No symbol) [0x00007FF64DF07023]
	(No symbol) [0x00007FF64DECFF5E]
	(No symbol) [0x00007FF64DED11E3]
	GetHandleVerifier [0x00007FF64E3D425D+3490733]
	GetHandleVerifier [0x00007FF64E3EBA43+3586963]
	GetHandleVerifier [0x00007FF64E3E147D+3544525]
	GetHandleVerifier [0x00007FF64E14C9DA+838442]
	(No symbol) [0x00007FF64DFFD04F]
	(No symbol) [0x00007FF64DFF9614]
	(No symbol) [0x000

'253425'